# Iceberg Recognition from Salelite Images Using Keras Deep NN

In this notebook I am going to develop a Deep Neural Network to recognise Iceberg in ocean from satelite images . For this image recognition problem I am going to use Keras deep learning features . Keras is high level deep NN library which can be built over a tensorflow or theano backend . Keras provides much more abstraction and flexibilty to built deep networks in a very simple way . 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

sample_submission.csv
test.json
train.json



The data for this problem is a image data in json format . We have to read the json data using pd.read_json( ) and we see it has two bands or channels and 1604 images in the training data .  The data can be downloaded form this [link](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge/data) .  

In [2]:
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')
train_df.head()

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


We need to preprocess the data to reshape it to make it in shape (1604,75,75,2) . We get the the training data in X_band .

In [3]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train_df["band_2"]])

In [4]:
X_band = np.zeros([1604,75,75,2])
for t in range(1604):
    X_band[t,:,:,0] = X_band_1[t]
    X_band[t,:,:,1] = X_band_2[t]

Now we import all required keras packages . 

In [5]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


The deep NN model in keras can be built in 2 ways . One is using Sequential API and the other is Model API . I am using Model API . So we built the model thatconsistes of 3 layers of convolution layer and two fully connected layer . Each conv layer is made of (Conv2D,BatchNormalization,ReLU,MaxPooling2D) block. 

In [6]:
def Iceberg_model(input_shape):
    X_in = Input(input_shape)
    
    X = Conv2D(10,kernel_size=(5,5),input_shape=(75,75,2))(X_in)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Conv2D(10,kernel_size=(5,5))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Conv2D(10,kernel_size=(5,5))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2,2))(X)
    
    X = Flatten()(X)
    X = Dense(50)(X)
    X = Activation('relu')(X)
    
    X = Dense(1)(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=X_in,outputs=X,name='Iceberg_model')
    return model

In Keras we have follow 4 simple steps to built a NN model . 
1.  create the model .
2.  compile the model .
3.  fit the model on the training data. 
4.  evaluate the model on the test data . 

For  creating the model we provide the shape of each training data (75,75,2) .

In [15]:
IcebergModel = Iceberg_model((75,75,2))

Compile the model using 'adam' optimizer , 'binary_crossentropy' loss and accuracy metric .

In [16]:
IcebergModel.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Now we fit the model with the training data having epochs = 20 and random batch_size = 128 . 

In [17]:
target = train_df['is_iceberg'].values
IcebergModel.fit(x=X_band,y=target,epochs=20,batch_size=128)

Epoch 1/20
1604/1604 [==============================] - 13s 8ms/step - loss: 0.6654 - acc: 0.6241
Epoch 2/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.4797 - acc: 0.7550
Epoch 3/20
1604/1604 [==============================] - 11s 7ms/step - loss: 0.3634 - acc: 0.8416
Epoch 4/20
1604/1604 [==============================] - 11s 7ms/step - loss: 0.3080 - acc: 0.8622
Epoch 5/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.2684 - acc: 0.8840
Epoch 6/20
1604/1604 [==============================] - 11s 7ms/step - loss: 0.2278 - acc: 0.9146
Epoch 7/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.1981 - acc: 0.9296
Epoch 8/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.1671 - acc: 0.9458
Epoch 9/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.1536 - acc: 0.9445
Epoch 10/20
1604/1604 [==============================] - 12s 7ms/step - loss: 0.1179 - acc: 0.9688
Epoch 11/20
1604/16

Evaluating the model on the traning data and it gets a 96.5 % accuarcy . 

In [18]:
IcebergModel.evaluate(x=X_band,y=target)

1604/1604 [==============================] - 6s 4ms/step


[0.090052791218507913, 0.96508728179551118]

Classification report on the training data .  

In [25]:
from sklearn.metrics import classification_report
pred_label = IcebergModel.predict(x=X_band)
pred_label[pred_label>0.5]=1
pred_label[pred_label<=0.5]=0
print(classification_report(target,pred_label))

             precision    recall  f1-score   support

          0       1.00      0.93      0.97       851
          1       0.93      1.00      0.96       753

avg / total       0.97      0.97      0.97      1604

